# MODELO DCGAN ----- PROBLEMA DE MODELIZACIÓN DE EMPRESAS GMV


El modelo de DCGAN que se muestra a continuación se trata de una modificación del que construyó Radford en el paper que se muestra a continuación. En el apartado del generador se expondrán las diferencias que presenta nuestro modelo con el suyo, a nivel arquitectónico. El paper en cuestión se puede encontrar en el siguiente link: [Paper original](https://arxiv.org/abs/1511.06434).

En lo que sigue, se procede a mostrar la implementación en python de este modelo y su aplicación a un conjunto de imágenes que habían sido previamente tratadas mediante modelos de Data Augmentation.

### IMPORTACIÓN DE ALGUNAS DE LAS LIBRERÍAS NECESARIAS

In [ ]:
import os
import time
import tensorflow as tf
#import tensorflow.compat.v1 as tf----- Estas dos lineas solo si se quiera ejecutar en un entorno que no tenga tf1. Poco recomendable, tarda mucho más.
                                        #Preferible hacerlo en un entorno que reconozca tf1, como kaggle, por ejemplo.
#tf.disable_v2_behavior()
import numpy as np
from glob import glob
import datetime
import random
import PIL
from PIL import Image
import matplotlib.pyplot as plt
from tqdm import tqdm_notebook as tqdm
%matplotlib inline
import urllib
import tarfile
import xml.etree.ElementTree as ET
from imageio import imread, imsave, mimsave
import shutil
import cv2
import glob
from imageio import imread, imsave, mimsave

## CODIFICACIÓN DE LOS ELEMENTOS QUE CONFORMAN LA ARQUITECTURA DE LA RED

### Generador

### VAMOS A COMENTAR LAS FUNCIONES QUE HEMOS UTILIZADO PARA CREAR EL GENERERADOR:

**tf.variable_scope()**: Crea un entorno que permite administrar variables. Sus argumentos:

    -'generator': El nombre del entorno
    
    - reuse=not training: Decide si las variables del entorno se reutilzarán o crearán de nuevo. Su valor es el contrario del valor booleano training, un argumento del generador.

**tf.layers.dense()**: Crea una capa completamente conectada. Sus argumentos:

    - z: El argumento de entrada
    
    - 4*4*512: Número de unidades de la capa
    
**tf.reshape()**: Prepara la capa para ser introducida en la primera capa convolucional. Sus argumentos:

    - fully_connected: Denota la capa que se quiere modelar
    
    -(-1, 4, 4, 512): Modfifica la dimensionalidad del tensor para adaptarlo a la capa convolucional.
    
    -1: Se utiliza para qur tf infiera esta dimensión para que el conjunto de elementos del tensor se mantenga constante.
    
**tf.nn.leaky_relu()**: La función de activación de la capa. Sus argumentos:

    -fully_connected: El sujeto de la activación.
    
    -Leaky Relu: La función de activación
    
**tf.layers.conv2d_transpose()**: Crea una capa de deconvolución. Sus argumentos:

    -fully_connected: El tensor de entrada al que se le va a aplicar la capa convolucional.
    
    -filtros: Especifica el número de canales que va a tener la capa. A lo largo de todo el generador va a denotar la myor dimensión de la siguiente capa convolucional, p.e.: para la primera 256.
    
    - kernel_size: Denota el tamaño del kernel que se va a aplicar en la operación convolución. Se mantendrá en [5,5] en todo el problema.
    
    - strides: Denota cuánto se desplaza el kernel en cada convolución. Se mantendrá constante en [2,2], salvo en la capa de salida, que será [1,1].
    
    - padding="SAME": Indica la estrategia del borde de relleno. Usamos SAME para garantizar la dimensionalidad del tensor.
    
    - kernel_initializer: Indica la inicilización de los pesos en la capa convolucional. Este valor depende de una variable global predeefinida.
    
    - name="trans_conv1": Nombre de la capa

**tf.layers.batch_normalization()**: Aplica una capa de normalización por lotes. En el pdf se puede encontrar por qué es beneficioso utilizar este filtro. Sus argumentos:

    - input: la capa a ser normalizada.
    
    - training: Valor booleano que dicta el comportamiento de la normalización por lotes: si training es True, entonces se utiliza la estadística del lote actual, mientras que si es False, se utilizan las de lotes pasados.
    
    - epsilon=EPSILON: Valor (pequeño) que se suma a la varianza del lote. Depende de una variable global predefinida.
    
    - name: El nombre

Por último, para la función de activación final (que rige el ouput), hemos decidido utilizar la función tangente hiperbólica usual.

In [ ]:

def generator(z, output_channel_dim, training):
    with tf.variable_scope("generator", reuse= not training):

        # 4x4x512
        fully_connected = tf.layers.dense(z, 4*4*512)
        fully_connected = tf.reshape(fully_connected, (-1, 4, 4, 512))
        fully_connected = tf.nn.leaky_relu(fully_connected)

        # 4x4x512 -> 8x8x256
        trans_conv1 = tf.layers.conv2d_transpose(inputs=fully_connected,
                                                 filters=256,
                                                 kernel_size=[5,5],
                                                 strides=[2,2],
                                                 padding="SAME",
                                                 kernel_initializer=tf.truncated_normal_initializer(stddev=WEIGHT_INIT_STDDEV),
                                                 name="trans_conv1")
        batch_trans_conv1 = tf.layers.batch_normalization(inputs = trans_conv1,
                                                          training=training,
                                                          epsilon=EPSILON,
                                                          name="batch_trans_conv1")
        trans_conv1_out = tf.nn.leaky_relu(batch_trans_conv1,
                                           name="trans_conv1_out")

        # 8x8x256 -> 16x16x128
        trans_conv2 = tf.layers.conv2d_transpose(inputs=trans_conv1_out,
                                                 filters=128,
                                                 kernel_size=[5,5],
                                                 strides=[2,2],
                                                 padding="SAME",
                                                 kernel_initializer=tf.truncated_normal_initializer(stddev=WEIGHT_INIT_STDDEV),
                                                 name="trans_conv2")
        batch_trans_conv2 = tf.layers.batch_normalization(inputs = trans_conv2,
                                                          training=training,
                                                          epsilon=EPSILON,
                                                          name="batch_trans_conv2")
        trans_conv2_out = tf.nn.leaky_relu(batch_trans_conv2,
                                           name="trans_conv2_out")

        # 16x16x128 -> 32x32x64
        trans_conv3 = tf.layers.conv2d_transpose(inputs=trans_conv2_out,
                                                 filters=64,
                                                 kernel_size=[5,5],
                                                 strides=[2,2],
                                                 padding="SAME",
                                                 kernel_initializer=tf.truncated_normal_initializer(stddev=WEIGHT_INIT_STDDEV),
                                                 name="trans_conv3")
        batch_trans_conv3 = tf.layers.batch_normalization(inputs = trans_conv3,
                                                          training=training,
                                                          epsilon=EPSILON,
                                                          name="batch_trans_conv3")
        trans_conv3_out = tf.nn.leaky_relu(batch_trans_conv3,
                                           name="trans_conv3_out")


        # 32x32x64 -> 64x64x32
        trans_conv4 = tf.layers.conv2d_transpose(inputs=trans_conv3_out,
                                                 filters=32,
                                                 kernel_size=[5,5],
                                                 strides=[2,2],
                                                 padding="SAME",
                                                 kernel_initializer=tf.truncated_normal_initializer(stddev=WEIGHT_INIT_STDDEV),
                                                 name="trans_conv4")
        batch_trans_conv4 = tf.layers.batch_normalization(inputs = trans_conv4,
                                                          training=training,
                                                          epsilon=EPSILON,
                                                          name="batch_trans_conv4")
        trans_conv4_out = tf.nn.leaky_relu(batch_trans_conv4,
                                           name="trans_conv4_out")

        # 64x64x32 -> 64x64x3
        logits = tf.layers.conv2d_transpose(inputs=trans_conv4_out,
                                            filters=3,
                                            kernel_size=[5,5],
                                            strides=[1,1],
                                            padding="SAME",
                                            kernel_initializer=tf.truncated_normal_initializer(stddev=WEIGHT_INIT_STDDEV),
                                            name="logits")
        out = tf.tanh(logits, name="out")
        return out

### Discriminador

### La red discriminadora sigue los mismos argumentos que la generadora, con la salvedad de que la función de output es una sigmoide. La explicación sobre la arquitectura de ambas redes se puede encontrar en el pdf.

In [ ]:
def discriminator(x, reuse):
    with tf.variable_scope("discriminator", reuse=reuse):

        # 64x64x3 -> 32x32x32
        conv1 = tf.layers.conv2d(inputs=x,
                                 filters=32,
                                 kernel_size=[5,5],
                                 strides=[2,2],
                                 padding="SAME",
                                 kernel_initializer=tf.truncated_normal_initializer(stddev=WEIGHT_INIT_STDDEV),
                                 name='conv1')
        batch_norm1 = tf.layers.batch_normalization(conv1,
                                                    training=True,
                                                    epsilon=EPSILON,
                                                    name='batch_norm1')
        conv1_out = tf.nn.leaky_relu(batch_norm1,
                                     name="conv1_out")

        # 32x32x32 -> 16x16x64
        conv2 = tf.layers.conv2d(inputs=conv1_out,
                                 filters=64,
                                 kernel_size=[5, 5],
                                 strides=[2, 2],
                                 padding="SAME",
                                 kernel_initializer=tf.truncated_normal_initializer(stddev=WEIGHT_INIT_STDDEV),
                                 name='conv2')
        batch_norm2 = tf.layers.batch_normalization(conv2,
                                                    training=True,
                                                    epsilon=EPSILON,
                                                    name='batch_norm2')
        conv2_out = tf.nn.leaky_relu(batch_norm2,
                                     name="conv2_out")

        # 16x16x64 -> 8x8x128
        conv3 = tf.layers.conv2d(inputs=conv2_out,
                                 filters=128,
                                 kernel_size=[5, 5],
                                 strides=[2, 2],
                                 padding="SAME",
                                 kernel_initializer=tf.truncated_normal_initializer(stddev=WEIGHT_INIT_STDDEV),
                                 name='conv3')
        batch_norm3 = tf.layers.batch_normalization(conv3,
                                                    training=True,
                                                    epsilon=EPSILON,
                                                    name='batch_norm3')
        conv3_out = tf.nn.leaky_relu(batch_norm3,
                                     name="conv3_out")

        # 8x8x128 -> 8x8x256
        conv4 = tf.layers.conv2d(inputs=conv3_out,
                                 filters=256,
                                 kernel_size=[5, 5],
                                 strides=[1, 1],
                                 padding="SAME",
                                 kernel_initializer=tf.truncated_normal_initializer(stddev=WEIGHT_INIT_STDDEV),
                                 name='conv4')
        batch_norm4 = tf.layers.batch_normalization(conv4,
                                                    training=True,
                                                    epsilon=EPSILON,
                                                    name='batch_norm4')
        conv4_out = tf.nn.leaky_relu(batch_norm4,
                                     name="conv4_out")

        # 8x8x256 -> 4x4x512

        conv5 = tf.layers.conv2d(inputs=conv4_out,
                                filters=512,
                                kernel_size=[5, 5],
                                strides=[2, 2],
                                padding="SAME",
                                kernel_initializer=tf.truncated_normal_initializer(stddev=WEIGHT_INIT_STDDEV),
                                name='conv5')
        batch_norm5 = tf.layers.batch_normalization(conv5,
                                                    training=True,
                                                    epsilon=EPSILON,
                                                    name='batch_norm5')
        conv5_out = tf.nn.leaky_relu(batch_norm5,
                                     name="conv5_out")

        flatten = tf.reshape(conv5_out, (-1, 4*4*512))
        logits = tf.layers.dense(inputs=flatten,
                                 units=1,
                                 activation=None)
        out = tf.sigmoid(logits)
        return out, logits

### Función de pérdida y optimizador

### VAMOS A COMENTAR LAS FUNCIONES QUE HEMOS UTILIZADO PARA CREAR LA FUNCIÓN DE PERDIDA:

**Variable g_model**: Utilizando el código del generador con los parámetros antes explicados que son input_z (vector de ruido) y el output_channel_dim qeu son los 3 colores del RGB.

**Variable noisy_input_real**: Creamos ruido en el input_real a través a través de la siguiente función:

    - tf.random_normal: Genera un valor aleatorio de una normal de n-dimensional.
    
**Variables d_model_real, d_logits_real**: Genera el discriminador con el input_real a través de noisy_input_real.
    
**Variables d_model_fake, d_logits_fake**: Genera el discriminador con el g_model(modelo generado a traves del generador con el input de ruido)

**tf.nn.sigmoid_cross_entropy_with_logits**: alcula la pérdida de entropía cruzada entre las predicciones (logits) y las etiquetas (targets) utilizando la función de activación sigmoide en el proceso. El resultado es un tensor que contiene los valores de pérdida para cada ejemplo en el lote de datos.

    -logits: Es un tensor que representa las predicciones antes de pasar por la función de activación sigmoide. Estos valores suelen ser salidas crudas del modelo, que pueden ser positivos o negativos.

    -targets: Es un tensor que contiene las etiquetas reales (etiquetas binarias) a comparar con las predicciones. Estas etiquetas son generalmente valores binarios (0 o 1) que indican si un ejemplo pertenece o no a una clase específica.

    -name (opcional): Un nombre opcional para la operación.

**tf.reduce_mean()**: Hace la media de los elementos de un tensor. Además del parámetro input_tensor tendremos los parámetros opcionales axis,keepdims y name que los mantenermos por defecto

    -input_tensor: El tensor de entrada al que se le va a aplicar, que en nuestro caso es el devuelto por la funcion anterior
    
**Variables d_loss_real y d_loss_fake**: Muestra como de bien el discriminador clasifica las imágenes reales como reales(d_loss_real) y las generadas como generadas(d_loss_fake).

**Variable g_loss(pérdida generativa)**: Muestra como de bien el generador esta realizando muestras reales que son difíciles de distinguir para el descriminador.

**Variable d_loss(pérdida del discriminador)**: Mide como de bien el discriminador puede distinguir entre los datos reales y los generados.(haciendo la media entre d_loss_real y d_loss_fake).

Por último, para la función de activación final (que rige el ouput), hemos decidido utilizar la función tangente hiperbólica usual.

In [ ]:
def model_loss(input_real, input_z, output_channel_dim):
    g_model = generator(input_z, output_channel_dim, True)

    noisy_input_real = input_real + tf.random_normal(shape=tf.shape(input_real),
                                                     mean=0.0,
                                                     stddev=random.uniform(0.0, 0.1),
                                                     dtype=tf.float32)

    d_model_real, d_logits_real = discriminator(noisy_input_real, reuse=False)
    d_model_fake, d_logits_fake = discriminator(g_model, reuse=True)

    d_loss_real = tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(logits=d_logits_real,
                                                                         labels=tf.ones_like(d_model_real)*random.uniform(0.9, 1.0)))
    d_loss_fake = tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(logits=d_logits_fake,
                                                                         labels=tf.zeros_like(d_model_fake)))
    d_loss = tf.reduce_mean(0.5 * (d_loss_real + d_loss_fake))
    g_loss = tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(logits=d_logits_fake,
                                                                    labels=tf.ones_like(d_model_fake)))
    return d_loss, g_loss

### VAMOS A COMENTAR LAS FUNCIONES QUE HEMOS UTILIZADO PARA CREAR EL OPTIMIZER:

**tf.trainable_variables()**: Se utiliza para tener un listado de todas las variables entrenables.

**g_vars**: variables relacionadas con el generador

**d_vars**: variables relacionadas con el discriminador
    
**tf.GraphKeys.UPDATE_OPS** es una clave utilizada en TensorFlow para agrupar operaciones en un grafo de cómputo que están relacionadas con la actualización de variables, como las operaciones de normalización o de regularización.
    
**gen_updates**: Representa las actualizaciones o operaciones que se aplican al generador durante el proceso de entrenamiento de la GAN.

**tf.control_dependencies()**: es una función en TensorFlow que se utiliza para controlar el orden de ejecución de las operaciones en un grafo de cómputo. En este caso queremos que se ejecuten primero las del generador.

**tf.train.AdamOptimizer()**: El optimizador Adam es un algoritmo de optimización ampliamente utilizado en el aprendizaje profundo para ajustar los pesos y sesgos de un modelo durante el proceso de entrenamiento.

    -learning_rate: La tasa de aprendizaje, que controla el tamaño de los pasos que el optimizador toma durante el proceso de entrenamiento.

    -beta1: El valor de beta 1 en el algoritmo de momentos de primer orden. Controla la contribución del momento del primer orden (media móvil exponencial del gradiente).
    
**Variables d_train_opt**: Es la variable refererida al optimizador(en nuestro casos Adam) para entrenar el discriminador. Esto lo hacemos minimizando la variable d_loss que es la perdida del discriminador y con la lista d_vars. Al optimizador le pasamos los siguientes parámetros:
    -LR_D = 0.0005
    -BETA1_D = 0.5

**Variable g_train_opt**: Es la variable refererida al optimizador(en nuestro casos Adam) para entrenar el generador. Esto lo hacemos minimizando la variable g_loss que es la perdida del generador y con la lista dg_vars. Al optimizador le pasamos los siguientes parámetros:
    -LR_G = 2e-4
    -BETA1_G = 0.5

In [ ]:
def model_optimizers(d_loss, g_loss):
    t_vars = tf.trainable_variables()
    g_vars = [var for var in t_vars if var.name.startswith("generator")]
    d_vars = [var for var in t_vars if var.name.startswith("discriminator")]

    update_ops = tf.get_collection(tf.GraphKeys.UPDATE_OPS)
    gen_updates = [op for op in update_ops if op.name.startswith('generator')]

    with tf.control_dependencies(gen_updates):
        d_train_opt = tf.train.AdamOptimizer(learning_rate=LR_D, beta1=BETA1_D).minimize(d_loss, var_list=d_vars)
        g_train_opt = tf.train.AdamOptimizer(learning_rate=LR_G, beta1=BETA1_G).minimize(g_loss, var_list=g_vars)
    return d_train_opt, g_train_opt

### **model_inputs()** es una función que te devuelve los inputs del modelo (definiéndolos primero) a partir de la dimensión del vector de ruido y de las dimensiones del data set (lo que posteriormente llamaremos imgsIn).

In [ ]:
def model_inputs(real_dim, z_dim):
    inputs_real = tf.placeholder(tf.float32, (None, *real_dim), name='inputs_real')
    inputs_z = tf.placeholder(tf.float32, (None, z_dim), name="input_z")
    learning_rate_G = tf.placeholder(tf.float32, name="lr_g")
    learning_rate_D = tf.placeholder(tf.float32, name="lr_d")
    return inputs_real, inputs_z, learning_rate_G, learning_rate_D

# Otras funciones de utilidad

### **Show samples** es una función que se utiliza en el training para ir mostrando la evolución del mismo. Sin emabargo, solo lo utilizamos cuando testeamos la red, ya que cuando la entrenamos evitamos usarla para mejorar los tiempos de ejecución.

In [ ]:
def show_samples(sample_images, name, epoch):
    figure, axes = plt.subplots(1, len(sample_images), figsize = (IMAGE_SIZE, IMAGE_SIZE))
    for index, axis in enumerate(axes):
        axis.axis('off')
        image_array = sample_images[index].astype('uint8')
        axis.imshow(image_array)
    plt.show()
    plt.close()

### **test** se utiliza para generar las imágenes de muestra que se muestran con **show_samples**, por lo que tampoco la vamos a utilizar.

In [ ]:
def test(sess, input_z, out_channel_dim, epoch):
    example_z = np.random.uniform(-1, 1, size=[SAMPLES_TO_SHOW, input_z.get_shape().as_list()[-1]])
    samples = sess.run(generator(input_z, out_channel_dim, False), feed_dict={input_z: example_z})
    sample_images = [((sample + 1.0) * 127.5).astype(np.uint8) for sample in samples]
    #show_samples(sample_images, IMG_DIR + "samples", epoch)

### **generate** es la función utilizada para la generación de los datos una vez la GAN ha sido entrenada. Toma como inputs la sesión de tf, los vectores de ruido z y la dimensión del canal de salida RGB (3).

Genera las imágenes utilizando el generador ya entrenado y los guarda en una lista, que luego convertiremos en un vector de numpy.

In [ ]:
def generate (sess, input_z, out_channel_dim):
    print ("Generando")
    imgs2=[]
    for i in tqdm(range(10)):
        example_z = np.random.uniform(-1, 1, size=[100, 100]).astype(np.float32)
        imgs = sess.run(generator(input_z, out_channel_dim, False), feed_dict={input_z: example_z})
        imgs = [((img + 1.0) * 127.5).astype(np.uint8) for img in imgs]
        for j in range(len(imgs)):
            imgs2.append(imgs[j])
    return imgs2

In [ ]:
def summarize_epoch(epoch, duration, sess, d_losses, g_losses, input_z, data_shape):
    minibatch_size = int(data_shape[0]//BATCH_SIZE)
    '''
    print("Epoch {}/{}".format(epoch, EPOCHS),
          "\nDuration: {:.5f}".format(duration),
          "\nD Loss: {:.5f}".format(np.mean(d_losses[-minibatch_size:])),
          "\nG Loss: {:.5f}".format(np.mean(g_losses[-minibatch_size:])))
    '''
    #fig, ax = plt.subplots()
    #plt.plot(d_losses, label='Discriminator', alpha=0.6)
    #plt.plot(g_losses, label='Generator', alpha=0.6)
    #plt.title("Losses")
    #plt.legend()
    #plt.savefig(OUTPUT_DIR + "losses_" + str(epoch) + ".png")
    #plt.show()
    #plt.close()
    test(sess, input_z, data_shape[3], epoch)

### **get_batches** es una función que, que como su nombre indica, toma los datos y genera los batches y los normaliza.

In [ ]:
def get_batches(data):
    batches = []
    for i in range(int(data.shape[0]//BATCH_SIZE)):
        batch = data[i * BATCH_SIZE:(i + 1) * BATCH_SIZE]
        augmented_images = []
        for img in batch:
            image = Image.fromarray(img.astype('uint8'))
            if random.choice([True, False]):
                image = image.transpose(Image.FLIP_LEFT_RIGHT)
            augmented_images.append(np.asarray(image))
        batch = np.asarray(augmented_images)
        normalized_batch = (batch / 127.5) - 1.0
        batches.append(normalized_batch)
    return batches

### Entrenamiento

### La función **train** toma como argumentos de entrada los **batches** con la función **get_bataches**, que trocea nuestro dataset, las dimensiones de dataset, el learning rate del generador (LR_G) y del discriminador (LR_D), que hemos fijado con valores muy bajos para poder captar las características del dataset de manera más rica. La estructura de la función train es la siguiente:

  - En primer lugar generamos los inputs del modelo con **model_inputs** y la función de pérdida y el optimizador con **model_loss** y **model_optimizers**.

  - Inicializamos la sesión de tensor flow, las épocas y las iteraciones, para a continuación movernos entre las épocas y los lotes para computar las funcciones de pérdida y actualizar los valores de los parámetros una vez se ha terminado el batch,.

  - Por último, una vez se hayan ejecutado todas las épocas, la GAN ya está esntrenada, por lo que simplemente llamamos a la función **generate** para que nos genere las imágenes que precisemos. Hemos ajustado los valores para que genere exactamente 1000 imágenes de cada tipo, aunque podríamos haberlo generado de manera arbitraria.

In [ ]:
def train(get_batches, data_shape, LR_G = 2e-4, LR_D = 0.0005):
    input_images, input_z, lr_G, lr_D = model_inputs(data_shape[1:], NOISE_SIZE)
    d_loss, g_loss = model_loss(input_images, input_z, data_shape[3])
    d_opt, g_opt = model_optimizers(d_loss, g_loss)
    generator_epoch_loss = 0
    train_d_losses = []
    train_g_losses = []
    generator_epoch_loss = 999

    with tf.Session() as sess:
        sess.run(tf.global_variables_initializer())
        epoch = 0
        iteration = 0
        d_losses = []
        g_losses = []

        for epoch in tqdm(range(EPOCHS)):
            epoch += 1
            start_time = time.time()

            for batch_images in get_batches:
                iteration += 1
                batch_z = np.random.uniform(-1, 1, size=(BATCH_SIZE, NOISE_SIZE))
                _ = sess.run(d_opt, feed_dict={input_images: batch_images, input_z: batch_z, lr_D: LR_D})
                _ = sess.run(g_opt, feed_dict={input_images: batch_images, input_z: batch_z, lr_G: LR_G})
                d_losses.append(d_loss.eval({input_z: batch_z, input_images: batch_images}))
                g_losses.append(g_loss.eval({input_z: batch_z}))

            summarize_epoch(epoch, time.time()-start_time, sess, d_losses, g_losses, input_z, data_shape)
            minibatch_size = int(data_shape[0]//BATCH_SIZE)
            generator_epoch_loss = np.mean(g_losses[-minibatch_size:])
            train_d_losses.append(np.mean(d_losses[-minibatch_size:]))
            train_g_losses.append(np.mean(g_losses[-minibatch_size:]))

            if epoch == EPOCHS:
                imgs=generate (sess, input_z, out_channel_dim=3)

    #fig, ax = plt.subplots()
    #plt.plot(train_d_losses, label='Discriminator', alpha=0.5)
    #plt.plot(train_g_losses, label='Generator', alpha=0.5)
    #plt.title("Training Losses")
    #plt.legend()
    #plt.savefig('train_losses.png')
    #plt.show()
    #plt.close()
    return imgs

**Inicialización de los hiperparámetros**

In [ ]:
IMAGE_SIZE = 64
NOISE_SIZE = 100
LR_D = 0.0005
LR_G = 2e-4
BATCH_SIZE = 64
EPOCHS = 100
BETA1_G = 0.5
BETA1_D = 0.5
WEIGHT_INIT_STDDEV = 0.02
MOMENTUM = 0.9
EPSILON = 0.0005
SAMPLES_TO_SHOW = 5 # each epoch

## PROCESAMIENTO DE LOS DATOS, ENTRENAMIENTO DE LA RED Y GENERACIÓN DE LOS NUEVOS DATOS SINTÉTICOS

**Librerías necesarias**

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import os
from sklearn.preprocessing import OneHotEncoder
import seaborn as sns
import matplotlib.pyplot as plt
import cv2
import matplotlib.pyplot as plt
import os
import numpy as np
import albumentations as A
import time
from tqdm import tqdm

### Recordamos el mapeo que habíamos hecho sobre las etiquetas, aunque no lo vayamos a utilizar.

In [ ]:
#RECORDAMOS EL DICCIONARIO CON EL 'mapeo' DE LOS PARÁMETROS
map_labels = {
    'Apple Braeburn':0,
  'Apple Granny Smith':1,
  'Apricot':2,
  'Avocado':3,
  'Banana':4,
  'Blueberry':5,
  'Cactus fruit':6,
  'Cantaloupe':7,
  'Cherry':8,
  'Clementine':9,
  'Corn':10,
  'Cucumber Ripe':11,
  'Grape Blue':12,
  'Kiwi':13,
  'Lemon':14,
  'Limes':15,
  'Mango':16,
  'Onion White':17,
  'Orange':18,
  'Papaya':19,
  'Passion Fruit':20,
  'Peach':21,
  'Pear':22,
  'Pepper Green':23,
  'Pepper Red':24,
  'Pineapple':25,
  'Plum':26,
  'Pomegranate':27,
  'Potato Red':28,
  'Raspberry':29,
  'Strawberry':30,
  'Tomato':31,
  'Watermelon':32
}

### Cargamos los datos

In [ ]:
import cv2
import numpy as np
import os, warnings

#VAMOS A IMPORTAR NUESTROS DATOS
file_path = '/kaggle/input/data-sergio-npz/data_tres_transformaciones.npz'


with np.load(file_path) as data:
    # Acceder a los datos del archivo .npz por nombre
    trainX = data['data']
    trainY = data['labels'] # cargar el array con los valores de las etiqeutas

# Trasnformar en arrays de numpy para mayor eficiencia
trainX = np.array(trainX)
trainY = np.array(trainY)
# Mostrar dimension del conjunto de muestras total
print("Forma de vector trainX de muestras:", trainX.shape)
print("Forma de vector trainY de etiquetas:", trainY.shape)

Forma de vector trainX de muestras: (33000, 100, 100, 3)
Forma de vector trainY de etiquetas: (33000,)


### Definimos algunas funciones útiles que vamos a necesitar para el procesamiento de los datos

In [ ]:
#VAMOS A DEFINIR ALGUNAS FUNCIONES QUE NOS VAN A SER UTILES EN LA GENERACION DE ESTAS NUEVAS IMAGENES
def extraer (num_label,trainX,trainY):
  lista=[]
  for i in range (0,len(trainX)):
    if trainY[i]==num_label:
      lista.append(trainX[i])
  return lista


#REESCALAMOS PARA QUE NOS QUEDE 64X64:
def reescalado_64(trainX):
  nuevo_trainX=[]
  for i in range (0,len(trainX)):
    imagen=trainX[i]
    canal_r = cv2.resize(imagen[:,:,0], (64, 64))
    canal_g = cv2.resize(imagen[:,:,1], (64, 64))
    canal_b = cv2.resize(imagen[:,:,2], (64, 64))
    nuevo_trainX.append(list(np.stack((canal_r, canal_g, canal_b), axis=-1)))
  return nuevo_trainX

#UN REESCALADO PARA QUE TODOS ESTÉN ENTRE -1 Y 1, AL FINAL NO HA SIDO UTILIZADO
def normalizacion(images):
  images_rescaled=(images-np.min(images)*np.ones_like(images))/(255-np.min(images))
  return images_rescaled

### Creamos una función que, dado el número asociado a una etiqueta y los datos, me entrena a la GAN y me genera tantos datos como quiera, como ya hemos mencionado serán unos 1000 por etiqueta.

In [ ]:
def generator_dcgan(num_label,trainX,trainY):
  #HACEMOS LA EXTRACCIÓN DE LOS DATOS Y SU REESCALADO INICIAL
  imagesIn=np.array(extraer(num_label,trainX,trainY))
  imagesIn=np.array(reescalado_64(imagesIn))
  imagesIn.shape
  #AQUI EL BUCLE DE ENTRENAMIENTO:
  start = time.time()

  print(">> Start training...")
  with tf.Graph().as_default():
    imgs=train(get_batches(imagesIn), imagesIn.shape)
  print(">> train time = ",time.time() - start)
  #TIENE QUE SER UNA LISTA, PARA PODER RESHAPEARLO. SE SUPONE QUE LO ES, PERO POR SI ACASO
  imgs=list(imgs)
  #HACEMOS EL RESHAPE:
  for i in range (len(imgs)):
    imgs[i]= cv2.resize(imgs[i], (100, 100))
  imgs=np.array(imgs)

  return imgs



### Creamos el bucle que nos genere los datos para todas las etiquetas

In [ ]:
#VAMOS A SACAR TODAS LAS IMÁGENES PARA TODAS LAS FRUTAS
frutas_generadas=[]
etiquetas=[]
n_class=33
for num_label in range (0,n_class):
    etiquetas=np.array(etiquetas)
    print(f'Vamos por la iteración {num_label}')
    frutas_generadas.append(generator_dcgan(num_label,trainX,trainY))
    nueva_etiqueta=np.ones(len(frutas_generadas[num_label]))*num_label
    etiquetas=np.concatenate([etiquetas,nueva_etiqueta])
print('Generación finalizada')

Vamos por la iteración 0
>> Start training...


  0%|          | 0/10 [00:00<?, ?it/s]

Generando



100%|██████████| 100/100 [03:52<00:00,  5.37s/it]A


>> train time =  239.04094648361206
Vamos por la iteración 1
>> Start training...


...

>> train time =  245.4386088848114
Vamos por la iteración 32
>> Start training...


  0%|          | 0/10 [00:00<?, ?it/s]

Generando



100%|██████████| 100/100 [04:01<00:00,  5.86s/it]A


>> train time =  244.34806323051453
Generación finalizada


### Guardamos los datos en un archivo de extensión .npz para poder exportarlos y evaluarlos en el clasificador.

In [ ]:
#LAS CONVERTIMOS EN ARRAYS NUEVAMENTE
frutas_generadas=np.array(frutas_generadas)
etiquetas=np.array(etiquetas)
frutas_final=[]
for i in range (len(frutas_generadas)):
    for j in range (0,len(frutas_generadas[i])):
        frutas_final.append(frutas_generadas[i,j])
frutas_final=np.array(frutas_final)
print(frutas_final.shape,etiquetas.shape,trainX.shape)

#LOS JUNTAMOS CON LAS QUE NOS HAN DADO DE INICIO
array_final_datos=np.concatenate([trainX,frutas_final])
array_final_etiquetas=np.concatenate([trainY,etiquetas])

#LO EXPORTAMOS TODO COMO UN UNICO ARCHIVO .npz, QUE TIENE LA MISMA ESTRUCTURA QUE EL ARCHIVO DE ENTRADA
np.savez('datos_gan',data=array_final_datos,labels=array_final_etiquetas)

(33000, 100, 100, 3) (33000,) (33000, 100, 100, 3)
